In [1]:
import pandas as pd
from random import randrange, uniform
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score,f1_score,accuracy_score
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv(r"D:\0AAAAAAAAAAAAAAAAAsmalldeskbook\神经网络与深度学习/data_new.csv",index_col = 0)
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,V12,V14,V16,V17,V18,V19,V23,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.363787,0.090794,-0.551600,-0.617801,-0.311169,-0.470401,0.207971,0.025791,0.403993,-0.110474,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,-0.255425,-0.166974,1.612727,1.065235,-0.143772,0.463917,-0.114805,-0.183361,-0.145783,0.101288,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,-1.514654,0.207643,0.624501,0.066084,-0.165946,-2.890083,1.109969,-0.121359,-2.261857,0.909412,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,-1.387024,-0.054952,-0.226487,0.178228,-0.287924,-1.059647,-0.684093,1.965775,-1.232622,-0.190321,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,0.817739,0.753074,-0.822843,0.538196,-1.119670,-0.451449,-0.237033,-0.038195,0.803487,-0.137458,69.99,0


In [3]:
df = df.drop(['Time','Amount'],axis = 1)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,V12,V14,V16,V17,V18,V19,V23,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.363787,0.090794,-0.551600,-0.617801,-0.311169,-0.470401,0.207971,0.025791,0.403993,-0.110474,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,-0.255425,-0.166974,1.612727,1.065235,-0.143772,0.463917,-0.114805,-0.183361,-0.145783,0.101288,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,-1.514654,0.207643,0.624501,0.066084,-0.165946,-2.890083,1.109969,-0.121359,-2.261857,0.909412,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,-1.387024,-0.054952,-0.226487,0.178228,-0.287924,-1.059647,-0.684093,1.965775,-1.232622,-0.190321,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,0.817739,0.753074,-0.822843,0.538196,-1.119670,-0.451449,-0.237033,-0.038195,0.803487,-0.137458,0


In [4]:
def SMOTE(sample: np.array, N: int, k: int) -> np.array:
    
    T, num_attrs = sample.shape
    
    # If N is less than 100%, randomize the minority class samples as only a random percent of them will be SMOTEd
    if N < 100:
        T = round(N / 100 * T)
        N = 100
    # The amount of SMOTE is assumed to be in integral multiples of 100
    N = int(N / 100)
    synthetic = np.zeros([T * N, num_attrs])
    new_index = 0
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(sample.values)
    def populate(N, i, nnarray):
        
        nonlocal new_index
        nonlocal synthetic
        nonlocal sample
        while N != 0:
            nn = randrange(1, k+1)
            for attr in range(num_attrs):
                dif = sample.iloc[nnarray[nn]][attr] - sample.iloc[i][attr]
                gap = uniform(0, 1)
                synthetic[new_index][attr] = sample.iloc[i][attr] + gap * dif
            new_index += 1
            N = N - 1
    
    for i in range(T):
        nnarray = nbrs.kneighbors(sample.iloc[i].values.reshape(1, -1), return_distance=False)[0]
        populate(N, i, nnarray)

    return synthetic

In [5]:
# 取出标签和特征
X = df.drop(['Class'], axis=1)
y = df['Class']
# 拆分出测试集合验证集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
neg_x_train = X_train[y_train == 1]
print("原来数据的欺诈样本:",neg_x_train.shape)
# 增强出来的欺诈样本
synthetic = SMOTE(neg_x_train,N=200,k=5)
print('增强出来的欺诈数据:',synthetic.shape)
# 拼接增强的数据和原来的数据
print(X_train.shape)
X_train_new = np.concatenate([X_train,synthetic],axis=0)
y_train_new = np.concatenate([y_train,np.ones([synthetic.shape[0]])])

# 对数据处理
from keras.utils import np_utils
X_train = X_train_new.reshape((-1,1,17))
Y_train = np_utils.to_categorical(y_train_new)
X_test = X_test.values.reshape((-1,1,17))
Y_test = np_utils.to_categorical(y_test)


原来数据的欺诈样本: (394, 17)
增强出来的欺诈数据: (788, 17)
(227845, 17)


In [7]:
from keras import Sequential
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding,BatchNormalization,Add,concatenate,Flatten

model = Sequential()

model.add(LSTM(units=50,return_sequences=True,input_shape=(1,17)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

# model.add(LSTM(units=50,return_sequences=True))
# model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

# model.add(Dense(units=256))
# model.add(Dropout(0.2))
model.add(Dense(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=16))
model.add(Dropout(0.2))

model.add(Dense(units=2,activation="softmax"))
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])



In [8]:
model.fit(x=X_train, y=Y_train, batch_size=8, epochs=30)
#预测
y_pred=model.predict(X_test)

Epoch 1/30
28580/28580 [==============================] - 408s 13ms/step - loss: 0.0103 - accuracy: 0.9984
Epoch 2/30
28580/28580 [==============================] - 317s 11ms/step - loss: 0.0087 - accuracy: 0.9985
Epoch 3/30
28580/28580 [==============================] - 376s 13ms/step - loss: 0.0082 - accuracy: 0.9985
Epoch 4/30
28580/28580 [==============================] - 406s 14ms/step - loss: 0.0074 - accuracy: 0.9985
Epoch 5/30
28580/28580 [==============================] - 381s 13ms/step - loss: 0.0075 - accuracy: 0.9986
Epoch 6/30
28580/28580 [==============================] - 426s 15ms/step - loss: 0.0073 - accuracy: 0.9986
Epoch 7/30
28580/28580 [==============================] - 292s 10ms/step - loss: 0.0070 - accuracy: 0.9985
Epoch 8/30
28580/28580 [==============================] - 346s 12ms/step - loss: 0.0071 - accuracy: 0.9986
Epoch 9/30
28580/28580 [==============================] - 494s 17ms/step - loss: 0.0068 - accuracy: 0.9987
Epoch 10/30
28580/28580 [============

ValueError: Found input variables with inconsistent numbers of samples: [56962, 1]

In [23]:
y_test

43428     1
49906     0
29474     0
276481    0
278846    0
         ..
75723     0
252263    0
221246    0
81910     0
59490     0
Name: Class, Length: 56962, dtype: int64

In [22]:
y_pred[0]

array([[1.1108495e-02, 9.8889148e-01],
       [1.0000000e+00, 2.3783276e-12],
       [1.0000000e+00, 2.3798251e-12],
       ...,
       [1.0000000e+00, 2.3783412e-12],
       [1.0000000e+00, 2.3783458e-12],
       [1.0000000e+00, 2.3880687e-12]], dtype=float32)

In [24]:
new_y_pred=y_pred[0][:,1]
for i in range(len(new_y_pred)):
    if new_y_pred[i]>0.5:
        new_y_pred[i]=1
    else:
        new_y_pred[i]=0
matrix = confusion_matrix(y_test, new_y_pred)

In [26]:
print(matrix)
recall = recall_score(y_test, new_y_pred)
print('recall:',recall)
acc = accuracy_score(y_test, new_y_pred)
print('accuracy:',acc)
f1 = f1_score(y_test, new_y_pred)
print("f1_score: ",f1)

[[56845    19]
 [   18    80]]
recall: 0.8163265306122449
accuracy: 0.9993504441557529
f1_score:  0.8121827411167513
